## Lesson 36: Multiple Regression

For this lesson we will be exploring multiple regression. This is a regression model with more than one predictor. Again, there are two, non-mutually exclusive, ideas about regression. The first is to use regression to understand the relationship between the predictors, also called features, inputs, or independent variables and the response, also called output, or dependent variable. For our model we will have only one response. In this capacity, we often are interested in inference. The second use of regression models is for prediction. Here we are only concerned with the predictive performance of the model and not the functional relationship between the inputs and the output.  This is how linear regression is considered a part of the machine learning ecology, as a supervised, model-based, batch learning method.   

Let's start by importing our packages and performing exploritory data analysis.

This notebook has been inspired from [Datacamp's](www.datacamp.com) Supervised Learning with scikit-learn course and the book Hands-On Machine Learning with Scikit-Learn & Tensorflow by Aurelien Geron.

### Import Packages
First we will import the usual packages

In [3]:
from datascience import *
import numpy as np
import pandas as pd
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', UserWarning)

Next we will import some new packages that you will start to learn. These include [scikit-learn](https://scikit-learn.org/stable/), [statsmodel](https://www.statsmodels.org/stable/index.html), and [seaborn](https://seaborn.pydata.org/). These should have been installed as part of the Anaconda distribution.

In [4]:
from sklearn import linear_model, datasets
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Motivation  

As future military officers and leaders, we need to appreciate our understanding and view of the world. Take the following test, http://forms.gapminder.org/s3/test-2018, without looking up answers, do it in earnest. Report your score in the next cell.

4/13, or 31%

### Data and EDA

In this notebook, you will be working with a subset of Gapminder data, see the [Gapminder](https://www.gapminder.org/) website for more information on this wonderful project by Hans Rosling and his team.  A subset of the data is provided to you in one CSV file called `gm_2008_region.csv`. Specifically, your goal will be to use this data to predict the life expectancy in a given country based on features such as the country's GDP, fertility rate, and population. This dataset has been heavily preprocessed to save you work.

Since the target variable, life expectancy, is quantitative, this is a regression problem. To begin, you will fit a linear regression with just one feature: `fertility`, which is the average number of children a woman in a given country gives birth to. In later exercises, you will use all the features to build regression models.

Before that, however, you need to import the data and get it into the form needed by `scikit-learn`. Then we will use `statsmodels` to get more detailed output. This involves creating feature and target variable arrays. Furthermore, since you are going to use only one feature to begin with, you need to do some reshaping using NumPy's `.reshape()` method. Don't worry too much about this reshaping right now, but it is something you will have to do occasionally when working with `scikit-learn` so it is useful to practice.

In [9]:
# Read the CSV file into a DataFrame: df
df = pd.read_csv('gm_2008_region.csv')

# Create arrays for features and target variable
y = df['life']
X = df['fertility']

# Print the dimensions of X and y before reshaping
print("Dimensions of y before reshaping: {}".format(y.shape))
print("Dimensions of X before reshaping: {}".format(X.shape))

# Reshape X and y
y = y.values.reshape(-1,1)
X = X.values.reshape(-1,1)

# Print the dimensions of X and y after reshaping
print("Dimensions of y after reshaping: {}".format(y.shape))
print("Dimensions of X after reshaping: {}".format(X.shape))

Dimensions of y before reshaping: (139,)
Dimensions of X before reshaping: (139,)
Dimensions of y after reshaping: (139, 1)
Dimensions of X after reshaping: (139, 1)


#### Exploring the Gapminder data

Start exploring the data using `pandas` methods such as `.info()`, `.describe()`, `.corr()`, and `.head()`.

Next let's look at some visual summaries of the data. The `pandas` package allows us to easily invoke the `matplotlib` to explore the data.

In [ ]:
df.hist(bins=11,figsize=(20,15));

In [ ]:
ax =sns.countplot(x='Region', data=df, palette='RdBu',orient="h")
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right");

In [ ]:
# Create a boxplot of life expectancy per region
df.boxplot('life','Region', rot=60);

In [ ]:
from pandas.tools.plotting import scatter_matrix

In [ ]:
scatter_matrix(df[df.columns[:8]],figsize=(15,15));

Next, we will construct a heatmap showing the correlation between the different features of the Gapminder dataset. Cells that are in green show positive correlation, while cells that are in red show negative correlation. Take a moment to explore this: Which features are positively correlated with life, and which ones are negatively correlated? Does this match your intuition?

The heatmap can be generated using Seaborn's heatmap function and the following line of code, where `df.corr()` computes the pairwise correlation between columns:

`sns.heatmap(df.corr(), square=True, cmap='RdYlGn')`

In [ ]:
sns.heatmap(df.corr(), square=True, cmap='RdYlGn');

### Fit and Predict a Single Variable

Now, you will fit a linear regression and predict life expectancy using just one feature. In this exercise, you will use the `fertility` feature of the Gapminder dataset. Since the goal is to predict life expectancy, the target variable here is `life`. 

First, create a scatter plot with `fertility` on the x-axis and `life` on the y-axis. As you will see, there is a strongly negative correlation, so a linear regression should be able to capture this trend. Your job is to fit a linear regression and then predict the life expectancy, overlaying these predicted values on the plot to generate a regression line. You will also compute and print the $R^2$ score using `sckit-learn's .score()` method.

In [ ]:
# Rename X so it makes more sense to us
X_fertility = X

plt.title('Linear Regression')
plt.scatter(X_fertility, y)
plt.xlabel('Fertility')
plt.ylabel('Life Expectancy');

To fit the model we need to create a `LinearRegression` instance from the `scikit-learn` package. This package has a consistent API, where from an instance we either create an estimator, a transform, or a predictor. To create an estimator, use the `.fit` method. See the [scikit-learn](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares) help page for more information.

In [ ]:
# Create the regressor: reg
reg = linear_model.LinearRegression()

# Create the prediction space
prediction_space = np.linspace(min(X_fertility), max(X_fertility)).reshape(-1,1)

# Fit the model to the data
reg.fit(X_fertility,y)

Next we will predict the response for the `prediction_space` we created above and plot the result.

In [ ]:
# Compute predictions over the prediction space: y_pred
y_pred = reg.predict(prediction_space)

# Plot regression line
plt.title('Linear Regression')
plt.scatter(X_fertility, y)
plt.xlabel('Fertility')
plt.ylabel('Life Expectancy')
plt.plot(prediction_space, y_pred, color='black', linewidth=3)
plt.show()

Let's get the coefficients and the $R^2$, the percent of variation in life expectancy explained by fertility.

In [ ]:
print('Intercept: ',np.round(reg.intercept_[0],4))
print('Slope: ',np.round(reg.coef_[0][0],4))
# Print R^2 
print("\n")
print("The coefficeint of determination:", np.round(reg.score(X_fertility, y),4))

The `scikit-learn` package is designed for production model building and is not intuitive for generating models in an interactive manner. The `statsmodels` package is better for this. It also allows us to use the `R` formula notation. In this case the dependent variables are on the left of the tilde and the predictors on the right. Let's model life expectancy as a function of fertility.

In [ ]:
# Fit regression model (using statsmodels)
results = smf.ols('life ~ fertility', data=df).fit()

# Inspect the results
print(results.summary())

This output has a great deal of information. The assumptions for this model are that the errors are independently distributed from a normal distribution with mean zero and constant variance. This means the slope coefficient is normally distributed as well, thus we can generate $p$-values.

The $F$-statistic is simultaneously testing if all coefficients are zero:    

$$H_{o}: \beta_1 = \beta_2 = \dots = \beta_n = 0 $$  
$$H_{a}: \mbox{At least one coefficient not 0}$$


The $t$-test is testing a single coefficient.  
$$H_{o}: \beta_i = 0 $$  
$$H_{a}: \beta_i \neq 0$$


We can check the normality plot using a quantile-quantile plot. This plot has the quantiles from a normal distribution on one axis and the empirical quantiles on the other. If it is normal, the data will be approximately on a straight line.

In [ ]:
sm.qqplot(results.resid,fit=True,line="45");

The normality assumption is suspect. We may want to use a bootstrap method instead.

### Multiple Regression  

We will now add more variables to the model. We will first take the approach of inference and then prediction.

In [ ]:
# Create model formula
"+".join(np.delete(df.columns,7))

In [ ]:
my_formula = "life~" +"+".join(np.delete(df.columns,7))
my_formula

In [ ]:
# Fit regression model (using statsmodels)
results2 = smf.ols(formula= my_formula , data=df).fit()

# Inspect the results
print(results2.summary())

Interpret the results. 

1. Why did the region get broken into 5 separate variables?
2. Why is $R^2$ and the $F$-statistic so significant, yet many individual $p$-values are not significant?
3. What is the predictive performance?

...

We can test subsets of the variables together using an F-test.

In [ ]:
print(results2.f_test("Region[T.East Asia & Pacific] = Region[T.Europe & Central Asia] = Region[T.Middle East & North Africa]=Region[T.South Asia] = Region[T.Sub-Saharan Africa] = 0"))

In [ ]:
print(results2.f_test("population = BMI_male = 0"))

In [ ]:
sm.qqplot(results2.resid,fit=True,line="45");

Run the model again, but drop region from the analysis.

In [ ]:
np.delete(df.columns,[7,9])

In [ ]:
my_formula2 = "life~" +"+".join(np.delete(df.columns,[7,9]))
my_formula2

In [ ]:
# Fit regression model (using statsmodels)
results3 = smf.ols(formula= my_formula2 , data=df).fit()

# Inspect the results
print(results3.summary())

In [ ]:
sm.qqplot(results3.resid,fit=True,line="45");

Continue adjusting the model. 

In [ ]:
my_formula3 = "life~" +"+".join(np.delete(df.columns,[0,1,7,9]))
my_formula3

In [ ]:
# Fit regression model (using statsmodels)
results4 = smf.ols(formula= my_formula3 , data=df).fit()

# Inspect the results
print(results4.summary())

In [ ]:
sm.qqplot(results4.resid,fit=True,line="45");

In [ ]:
from statsmodels.graphics.regressionplots import plot_leverage_resid2
fig, ax = plt.subplots(figsize=(8,6))
fig = plot_leverage_resid2(results4, ax = ax)

In [ ]:
from statsmodels.graphics.regressionplots import influence_plot
fig, ax = plt.subplots(figsize=(8,6))
fig = influence_plot(results4,ax=ax)

#### Creating dummy variables
`scikit-learn` does not accept non-numerical features. In `statsmodels` 'Region' was automatically taken care for us. We want to include 'Region' as it may contain features very useful in predicting life expectancy. For example, Sub-Saharan Africa has a lower life expectancy compared to Europe and Central Asia. Therefore, if you are trying to predict life expectancy, it would be preferable to retain the 'Region' feature. To do this, you need to binarize it by creating dummy variables, which is what you will do in this exercise.

In [ ]:
# Create dummy variables: df_region
df_region = pd.get_dummies(df)

# Print the columns of df_region
print(df_region.columns)

# Create dummy variables with drop_first=True: df_region
df_region = pd.get_dummies(df,drop_first=True)

# Print the new columns of df_region
print(df_region.columns)

#### Train/test split for regression
For a measure of prediction error it is best to use data that was not used in the building of the model. An easy first attempt is to break the data into train and test sets. This is vital to ensure that our supervised learning model is able to generalize well to new data. This is also true for classification models.

In this exercise, you will split the Gapminder dataset into training and testing sets, and then fit and predict a linear regression over all features. In addition to computing the $R^2$ score, you will also compute the Root Mean Squared Error (RMSE), which is another commonly used metric to evaluate regression models. The feature array `X` and target variable array `y` have been pre-loaded for you from the DataFrame `df`.

In [ ]:
df_region.head()

In [ ]:
df_region.drop(columns=["life"]).head()

In [ ]:
X=df_region.drop(columns=["life"])
X.shape

In [ ]:
# Import necessary modules
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=42)

# Create the regressor: reg_all
reg_all = LinearRegression()

# Fit the regressor to the training data
reg_all.fit(X_train,y_train)

# Predict on the test data: y_pred
y_pred = reg_all.predict(X_test)

# Compute and print R^2 and RMSE
print("R^2: {}".format(reg_all.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print("Root Mean Squared Error: {}".format(rmse))